#### Note To Self

As of now, when I run the tectonic earth simulations for too long, the resulting mountains get too big. Although I have made attempts at solving this issue in the past, I feel like erosion processes is what I should be using. I should modify the Gospl notebook so that a new gospl simulation is run after each tectonic earth simulation iteration, and see if the Gospl errosion algorithm prevents my mountains from growing too high.

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from scipy.interpolate import griddata
from EarthsAssistant import EarthAssist

In [ ]:
#Code to confirm that all code based on previous notebooks works as expected
if False:
    earth = Earth(
                startTime = 10,
                endTime = 0,
                deltaTime = 1,
                baseUplift = 2000,
                distTransRange = 1000000, 
                numToAverageOver = 10,
                earthRadius = 6378137.,
                useKilometres = False,
                useGospl = True)
    
    #Run earth simulation, run Gospl simulation and animate results
    earth.runTectonicSimulation()
    #gosplMan = GosplManager(earth, subdivisions=4)
    #gosplMan.createAllFilesAndRunSimulation()
    #gosplMan.animateGosplOutput()
    earth.animate(lookAtLonLat=[0, 0])

# Introduction

As of now, our earth simulation seems to look ok as it runs over shorter periods of time. However, various errors become obvious as we run the simulation for longer. The most prominent problem so far is that when two continents diverge, they don't leave ocean behind. Additionally, we don't have a mechanism for ocean floor formations. These are both phenomina related to tectonic plate divergence which we will work towards solving in this notebook.

This problem can be demonstrated clearly in the resulting animation bellow:

In [ ]:
%%HTML
<video width="800" height="608" controls>
  <source src="./NotebookFiles/DivergenceProblem.mp4" type="video/mp4">
</video>

``` python
#Code for generating the above video
if False:
    earth = Earth(
        startTime = 100,
        endTime = 0,
        deltaTime = 2,
        baseUplift = 2000,
        distTransRange = 1000000, 
        numToAverageOver = 10,
        earthRadius = 6378137.,
        useKilometres = False,
        useGospl = True),
        useDivergeLowering = False
    earth.runTectonicSimulation()
    earth.animate()
```

# Diverging Continents

In order to make diverging contents leave oceans behind, we need to lower vertices that are nearby diverging plate boundaries. In our code, we will refer to the amount of lowering as *divergeLowering*, which will depend on the following transfers:

- **Distance Transfer** - The distance that a vertex is to the closest diverging plate boundary. Vertices that are close to a plate boundary should be lowered more than vertices far away from the boundaries, and so we will pass the distances through a gaussian shaped function to calculate the distance contribution to diverge lowering. Given the mean $\mu$ and variance $\sigma$, our gaussian is defined by:

$$
Gaussian(x, \mu, \sigma) = exp \Big( \frac{- (x - \mu)^2}{\sigma} \Big)
$$

- **Height Transfer** - The current height of the vertex. Vertices that are on a higher elevation (on land) should be lowered more than vertices that have a lower elevation (in ocean). Therefore the height contribution will be calculated by passing the current heights of vertices through a sigmoid function. Given the centre $\mu$ and steepness $s$, our sigmoid is defined by:

$$
Sigmoid(x, \mu, s) = \frac{1}{1 + e^{- s (x - \mu)}}
$$

In the code bellow, we define these two functions and provide a simple plot of each. Note that the default parameters are chosen so that most of the function outputs varies at input values in the range $x \in [-1, 1]$.

In [ ]:
#Gaussian function used for profile of diverge lowering
def gaussian(x, mean=0, variance=0.25):
    return np.exp(-((x - mean)**2) / variance)

#Sigmoid function
def sigmoid(x, centre=-0.1, steepness=6):
    return 1 / (1 + np.exp(-(x - centre) * steepness))

#Plot the gaussian and sigmoid functions
x = np.arange(-2, 2, 0.01)
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
axs[0].plot(x, gaussian(x, mean=0, variance=0.25))
axs[0].title.set_text("Gaussian")
axs[0].set_xlabel('Distance Input')
axs[0].set_ylabel('Output')
axs[1].plot(x, sigmoid(x, centre=0, steepness=4))
axs[1].title.set_text("Sigmoid")
axs[1].set_xlabel('Height Input')
axs[1].set_ylabel('Output')

Now that we have define the transfer functions for diverge lowering, we need to get the appropriate heights and distance input values. Getting the heights input is easy, since all we have to do is read the current heights from *Earth.heights*. However, to get the distances from diverging plate boundaries, we first need to get the location of all diverging plate boundaries, which can be identified by having a negative collision speed (as discussed in notebook 2). The distances will then be calculated based on the boundary lines. Most of the code for this is already contained in the *PlateBoundaries.py* file.

The code bellow demonstrates how we will get our distances to diverging plate boundaries.

In [ ]:
#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(boundaries):
    divXYZ, divLinePoints = [], []
    for bound in boundaries.plateBoundaries:
        for i in range(bound.lineCentres.shape[0]):
            if bound.collisionSpeed[i] < 0:
                divXYZ.append(bound.lineCentres[i])
                divLinePoints.append(bound.linePoints[i])
    return np.array(divXYZ), np.array(divLinePoints)

#Get distance from vertices to diverging plate boundaries.
def getDistanceToDivergence(divXYZ, sphereXYZ, linePoints):
    distIds = cKDTree(divXYZ).query(sphereXYZ, k=1)[1]
    distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)
    return distToBound

#Create an earth object
earth = Earth(
    startTime = 10,
    endTime = 0,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True,
    useDivergeLowering=False)

#Create an instance of the Boundaries class (this code is normally called by earth.runTectonicSimulation())
time = earth.startTime - earth.deltaTime
plateIds = earth.getPlateIdsAtTime(time)
rotations = earth.getRotations(plateIds, time)
boundaries = Boundaries(time, earth, plateIds, rotations)

#Run the newly defined functions
divXYZ, divLinePoints = getDivergingBoundaries(boundaries)
distToDivs = getDistanceToDivergence(divXYZ, earth.sphereXYZ, divLinePoints)

#Visualize the results
plotter = pv.PlotterITK()
earthMesh = earth.getEarthMesh()
earthMesh['distToDivs'] = distToDivs**0.25
boundaryLines = Boundaries.getBoundaryLines(boundaries.plateBoundaries)
plotter.add_mesh(earthMesh, scalars='distToDivs')
plotter.add_mesh(boundaryLines, color='r')
plotter.show()

Using the data that we have just calculated, getting the diverge lowering is simple, as shown bellow:

In [ ]:
#Define function for diverge lowering
def getDivergeLowering(distToDivs, heights, maxDistance=200000, minMaxHeights=8000):
    distanceTransfer = gaussian(distToDivs / maxDistance)
    heightTransfer = sigmoid(heights / minMaxHeights)
    return distanceTransfer * heightTransfer

#Get diverge lowering
divergeLowering = getDivergeLowering(distToDivs, earth.heights)

#Visualize results
plotter = pv.PlotterITK()
earthMesh['divergeLowering'] = divergeLowering
plotter.add_mesh(earthMesh, scalars='divergeLowering')
plotter.add_mesh(boundaryLines, color='r')
plotter.show()

Now that we have gotten the diverge lowering property, we want to apply it at each of earth's simulation step. In the code bellow, the function *doSimulationStep()* is similar to *earth.doSimulationStep()*, but with diverge lowering also implemented in it. We then provide an animation of the results.

In [ ]:
#Given an earth object, apply diverge lowering to it
def applyDivergeLowering(earth, baseLowering=2000):
    divXYZ, divLinePoints = getDivergingBoundaries(earth.boundaries)
    distToDivs = getDistanceToDivergence(divXYZ, earth.sphereXYZ, divLinePoints)
    earth.heights -= baseLowering * getDivergeLowering(distToDivs, earth.heights)

#Function that is similar to earth.doSimulationStep() but with added code from this notebook
def doSimulationStep(earth, time):
    earth.heights = np.copy(earth.heightHistory[-1])
    earth.timeHistory.append(time - earth.deltaTime)
    earth.setPlateData(time)
    earth.heights += earth.boundaries.getUplifts()
    applyDivergeLowering(earth) #New line of code to be added in earth.doSimulationStep()
    earth.movePlatesAndRemesh()
    earth.createTectonicDisplacements()
    
#Create new earth object
earth = Earth(
    startTime = 20,
    endTime = 0,
    deltaTime = 1,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True,
    useDivergeLowering=False)

#Lopp that is similar to earth.runTectonicSimulation()
for time in earth.simulationTimes[:-1]:
    print('Currently simulating at {} Millions years ago'.format(time), end='\r')
    doSimulationStep(earth, time)

earth.animate(lookAtLonLat=[0, 0])

In [ ]:
%%HTML
<video width="800" height="608" controls>
  <source src="./TectonicSimulation.mp4" type="video/mp4">
</video>

# Sea Floor Spreading

The depth of oceans is primarily determined by the age of the ocean floors. This corrolation is made clear by comparing the heightmap of ocean floor and age map of ocean floors as shown bellow. So in order to simulated the evolution of ocean floor spreading, we will also need to keep track of the ocean floors age.

Note that the colormap in the pyvista visualization bellow shows ocean depths within the range of $[-5500, -2500]$, which is the depth range at which ocean floor spreading takes place.

In [ ]:
#Create new earth object
earth = Earth(startTime=1, endTime=0, deltaTime=1)

#Create mesh objects
earthMesh = earth.getEarthMesh()
earth.setPlateData(earth.startTime)
boundMesh = earth.boundaries.getBoundaryMesh()

#Create scalar mesh attributes to visualize ocean floor heights
oceanFloorHeights = earth.heights
oceanFloorHeights[oceanFloorHeights>-2500] = -2500
oceanFloorHeights[oceanFloorHeights<-5500] = -5500
earthMesh['oceanFloorHeights'] = oceanFloorHeights

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='oceanFloorHeights')
plotter.add_mesh(boundMesh)
plotter.show()

<div>
<img src="NotebookFiles/OceanFloorAge.jpg" width="800">
</div>

### Initial Age Estimates

Before we can keep track of the ocean floor's age as the simulation runs, we first need an initial estimate of the ocean floors age at the first iteration of the tectonic earth's simulation. Since we don't have any height data available of the ocean floors during the early years of our simulations, we will have to approximate it based on tectonic boundary locations and plate velocities data provided to us by *Gplates*.

To begin with, we create a function that gets the location and speed data of oceanic rift locations. Then, for each vertex on our sphere that corresponds to an ocean, it's distance $d$ to rift boundaries, and the speed of divergence $s$ of those nearest boundaries. This is done using similar methods as for subduction uplift. Since speed is generally defined by distance over time $s = \frac{d}{t}$, the age $t$ of an ocean floor can be estimated by:

$$
t = \frac{d}{s + \epsilon}
$$
where $\epsilon$ is a small constant used purely to avoid division by zero.


In [ ]:
#Get coordinates and line points of all diverging plate boundary locations
def getOceanicRifts(bounds):
    riftXYZ, riftLinePoints, riftSpeeds = [], [], []
    for bound in bounds.plateBoundaries:
        if bound.gpmlBoundType == 'gpml:MidOceanRidge':
            for i in range(bound.lineCentres.shape[0]):
                if bound.collisionSpeed[i] < 0:
                    riftXYZ.append(bound.lineCentres[i])
                    riftLinePoints.append(bound.linePoints[i])
                    riftSpeeds.append(bound.collisionSpeed[i])
    return np.array(riftXYZ), np.array(riftLinePoints), np.array(riftSpeeds)

#Get distance and speeds from the sphere's vertices to oceanic rifts
def getDistAndSpeedToRifts(earth, bounds):
    riftXYZ, riftLinePoints, riftSpeeds = getOceanicRifts(bounds)
    distIds = cKDTree(riftXYZ).query(earth.sphereXYZ, k=10)[1]
    distIds[distIds >= riftXYZ.shape[0]] = riftXYZ.shape[0]-1
    closestLinePoints = riftLinePoints[distIds]
    riftSpds = riftSpeeds[distIds]
    distToRift = Boundaries.getDistsToLinesSeg(earth.sphereXYZ, closestLinePoints[:, 0])
    riftSpds = np.mean(riftSpds, axis=1)
    return distToRift, riftSpds

#Approximate the initial ocean floor age based on distance and speed to oceanic rifts
def getOceanFloorAge(earth, bounds, ageMultiplier=2, avoidZeroDivision=10):
    distToRift, riftSpeeds = getDistAndSpeedToRifts(earth, bounds)
    age = -distToRift / (riftSpeeds + avoidZeroDivision)
    age[earth.heights>0] = np.max(age)
    return ageMultiplier * age
    
#Create new earth object
earth = Earth(startTime = 5, endTime = 0, deltaTime = 1)
earth.runTectonicSimulation()
earth.setPlateData(earth.startTime)
bounds = earth.boundaries

#Run the new functions for getting ocean floor age
oceanFloorAge = getOceanFloorAge(earth, bounds)

#Get mesh objects for plotting
earthMesh = earth.getEarthMesh()
boundaryMesh = bounds.getBoundaryMesh()
contour = earthMesh.contour([0], scalars='heights')
earthMesh['oceanFloorAge'] = oceanFloorAge

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='oceanFloorAge')
plotter.add_mesh(contour, color="peru", opacity=1.)
plotter.add_mesh(boundaryMesh)
plotter.show()

The depth of ocean floors will be based on the following equation:

$$
depth = h = min
\begin{cases}
    2500 + 350 \sqrt{t} \\
    5500
\end{cases}
$$

To visualize this, we define an function that for the above equation and visualize it's profile. Note that the maximum depth occours when the ocean floor is 75 million years old.

In [ ]:
def getOceanDepth(age, riftDepth=2500, ageConst=350, maxDepth=5500):
    depth = riftDepth + ageConst * np.abs(age)**0.5
    depth[depth>maxDepth] = maxDepth
    return - depth

age = np.arange(-100, 100, 0.1)
plt.figure(figsize=(16, 2))
plt.plot(age, getOceanDepth(age))
plt.show()

In the earlier years of our simulation, we won't have ocean topological data available, and so we will initiate the ocean floor depths based on the above function. To show that this equation is indeed a good approximate, we apply this equation to our estimates of the ocean floor age to get the following results.

In [ ]:
oceanDepth = getOceanDepth(oceanFloorAge)
oceanDepth[earth.heights>0] = np.max(oceanDepth)
flatEarthMesh = earth.getFlatEarthMesh()

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(flatEarthMesh, scalars=oceanDepth)
plotter.show()

To confirm that we do indeed have a good approximation of the ocean floor's age, we can compare the above visualization will the image bellow. Since our model produces a maximum ocean depth at an ocean age of 75 million years, the dark blue region in our plot above correspond to ocean ages of older than 75 million years, which corresponds to the green regions in the image bellow.

<div>
<img src="NotebookFiles/OceanFloorAge.jpg" width="1000">
</div>

### Simulating Ocean Floors Over Time

Once we have initialized the ocean floors and their corresponding age, we now wish to simulate the change in height over time. Taking the derivative of the above equation with respect to time for $t \in [0, 75]$ Mya, we get the change in height $dh$ given a change in time $dt$ as:

$$
d h = \frac{350}{2\sqrt{t}} dt
$$

Note that as time passes, the ocean floors will move along with the tectonic plates. So for the same reasons that we need to interpolate heights during the remesh, we will also need to interpolate ocean floor age during its remesh.

Since most of the code for interpolating the ocean floor age is similar to that of interpolating heights after moving tectonic plates, we will redefine the remesh algorithm in such a way so that we can more easily add more scalars (Eg. Heights or ocean floor age) to be remeshed later on.

As discussed in tutorial 1, we begin by moving the tectonic plates along the sphere, convert the sphere into cylinder to help identify regions of overlapping plates through clustering means. These clusters will now be stored as a class attribute so that we can re-use them to interpolate any scalar field that moves along with the plates (such as height, ocean age, crust thickness...). The code bellow does this clustering, and provides a visualization of the clusters.

In [ ]:
#Run algorithm for moving plates and remeshing the sphere
def movePlates(earth):
    movedEarthXYZ = earth.movePlates(earth.plateIds, earth.rotations)
    movedLonLat = EarthAssist.cartesianToPolarCoords(movedEarthXYZ)
    earth.movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)

#Create cylinder of earth such that vertices are all equally spaced apart
#Overriding plates can then be identified as vertices being relatively closer to each other
def createEarthCylinder(earth):
    phiRes = earth.phiResolution
    thetaRes = earth.thetaResolution
    movedLonLat = earth.movedLonLat
    northToSoutDist = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    cylinderRadius = thetaRes * northToSoutDist / (np.pi * phiRes * 2)
    cylinderXYZ = EarthAssist.cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    return cylinderXYZ, thetaRes   
    
#Set the parameters "earth.isCluster" and "earth.clusterPointsNeighboursId"
#These will represent regions of overriding/colliding plates, and will be used for interpolating scalars after a remesh
def setRemeshClusters(earth):
    cylinderXYZ, thetaRes = createEarthCylinder(earth)
    
    #Run the clustering algorithm
    threshHoldDist = earth.clusterThresholdProportion * 360 / thetaRes
    cluster = DBSCAN(eps=threshHoldDist, min_samples=earth.minClusterSize).fit(cylinderXYZ)
    earth.isCluster = (cluster.labels_ != -1)

    #Create KDTree to find nearest neighbours of each point in cluster
    pointsInClusterLonLat = cylinderXYZ[earth.isCluster]
    clusterKDTree = cKDTree(pointsInClusterLonLat).query(pointsInClusterLonLat, k=earth.numOfNeighbsForRemesh+1)
    earth.clusterPointsNeighboursId = clusterKDTree[1]

#Create new earth object
earth = Earth(startTime = 60, endTime = 0, deltaTime = 10, minClusterSize=2)
earth.oceanFloorAge = getOceanFloorAge(earth, earth.boundaries)

#Move plates and set remesh clusters
movePlates(earth)
setRemeshClusters(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
isCluster = earth.isCluster

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(pv.Sphere(radius=earth.earthRadius-200000))
plotter.add_points(movedEarthXYZ[isCluster], color='r')
plotter.add_points(movedEarthXYZ[isCluster==False], color='b')
plotter.show()

Now that we have stored these clusters as a class attribute, we can interpolate any scalar attributes by simply adding one lines of code in the *interpolateScalars()* function bellow for each scalar. Note that we could easily extend this code so that we can interpolate vector fields too, however so far I have not found a need to do so.

To help visualize results, we have provided an earth mesh with contour lines and mesh topology based on old heights, and mesh scalar colors corresponding to the new interpolated heights. We have attached some additional scalar attributes to the mesh, which can be visualized by selecting them in the drop down menu in the top left of the pyvista window.

Note that when instantiating the earth object, we selected a large *deltaTime* of 10 My to make plate movement very visible. In practice we will use a finer *deltaTime* to simulate finer temporal details.

In [ ]:
#Interpolate scalars after moving tectonic plates
def interpolateScalars(earth):
    setRemeshClusters(earth)
    #heightsForRemesh = prepareScalarsForRemesh(earth, np.copy(earth.heights))
    #floorAgeForRemesh = prepareScalarsForRemesh(earth, np.copy(earth.oceanFloorAge))
    newHeights = interpolateScalar(earth, np.copy(earth.heights))
    newFloorAge = interpolateScalar(earth, np.copy(earth.oceanFloorAge))
    return newHeights, newFloorAge

#Function based on interpolateHeights() from earth's remesh algorithm
def interpolateScalar(earth, scalar):
    scalarForRemesh = prepareScalarsForRemesh(earth, scalar)
    movedLonLat = earth.movedLonLat
    newScalar = griddata(movedLonLat, scalarForRemesh, earth.lonLat)
    whereNAN = np.argwhere(np.isnan(newScalar))
    newScalar[whereNAN] = griddata(movedLonLat, scalarForRemesh, earth.lonLat[whereNAN], method='nearest')
    return newScalar
    
#To prepare scalars for interpolation, we set scalars of overlaping vertices to the maximum of their neighbours
def prepareScalarsForRemesh(earth, scalars):
    isCluster = earth.isCluster
    clusterPointsNeighboursId = earth.clusterPointsNeighboursId
    scalarsInCluster = scalars[isCluster]
    neighbourScalars = scalarsInCluster[clusterPointsNeighboursId[:, 1:]]
    scalars[isCluster] = np.max(neighbourScalars, axis=1)
    return scalars

#Run the new functions for getting ocean floor age
newHeights, newFloorAge = interpolateScalars(earth)

#Get mesh objects for plotting
bounds = earth.boundaries
boundaryMesh = bounds.getBoundaryMesh()
earthMesh = earth.getEarthMesh()
earthMesh['newHeights'] = newHeights
earthMesh['oceanDepth'] = oceanDepth
earthMesh['newFloorAge'] = newFloorAge
earthMesh['oceanFloorAge'] = earth.oceanFloorAge
contour = earthMesh.contour([0], scalars='heights')

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='newHeights')
plotter.add_mesh(contour, color="peru", opacity=1.)
plotter.show()

In [ ]:
crash Python here

In [ ]:
earth = Earth(startTime = 60, endTime = 0, deltaTime = 10)
earth.doSimulationStep(earth.startTime)
earth.setPlateData(earth.startTime)
bounds = earth.boundaries

earthMesh = earth.getEarthMesh()
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh)
plotter.show()

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from EarthsAssistant import EarthAssist

#Want speeds to nearest diverging boundary
def getSpeedsToDivergence(divXYZ, sphereXYZ, linePoints):
    distIds = cKDTree(divXYZ).query(sphereXYZ, k=50)[1]
    distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)

#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(bounds):
    divXYZ, divLinePoints, divSpeeds = [], [], []
    for bound in bounds.plateBoundaries:
        if bound.boundType == 2:
            for i in range(bound.lineCentres.shape[0]):
                if bound.collisionSpeed[i] < 0:
                    divXYZ.append(bound.lineCentres[i])
                    divLinePoints.append(bound.linePoints[i])
                    divSpeeds.append(bound.collisionSpeed[i])
    return np.array(divXYZ), np.array(divLinePoints), np.array(divSpeeds)

#Create new earth object
earth = Earth(
    startTime = 5,
    endTime = 0,
    deltaTime = 1)
earthMesh = earth.getEarthMesh()
earth.setPlateData(earth.startTime)
bounds = earth.boundaries

#Run the getUplifts function so we can get it's speed transfer (and avoid having to calculate it twice)
bounds.getUplifts()
speedTransfer = bounds.speedTransfer

#Get distance to diverging plate boundaries
#distToDivs = bounds.setDistanceToDivergence()

divXYZ, divLinePoints, divSpeeds = getDivergingBoundaries(bounds)

distIds = cKDTree(divXYZ).query(earth.sphereXYZ, k=10)[1]
distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
closestLinePoints = divLinePoints[distIds]
distToBound = Boundaries.getDistsToLinesSeg(earth.sphereXYZ, closestLinePoints[:, 0])

divSpds = divSpeeds[distIds]
divSpds = np.mean(divSpds, axis=1)
#print(np.mean(divSpds, axis=1).shape)

In [ ]:
#gaussDeleteThis = EarthAssist.gaussian(distToBound/4000000)**0.5
contour = earthMesh.contour([0], scalars='heights')
boundaryMesh = bounds.getBoundaryMesh()


age = - distToBound / (divSpds + 10)
heights= earthMesh['heights']
heights[heights>0] = 0
heights = (-heights)**0.5
earthMesh['newHeights'] = heights


#cutOff = 200
#age[age>cutOff] = cutOff
age[earth.heights>0] = np.max(age)


#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars=age**0.5)
plotter.add_mesh(boundaryMesh)
plotter.add_mesh(contour, color="peru", opacity=1.)
plotter.show()

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from EarthsAssistant import EarthAssist


#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(bounds):
    divXYZ, divLinePoints, divSpeeds = [], [], []
    for bound in bounds.plateBoundaries:
        for i in range(bound.lineCentres.shape[0]):
            if bound.collisionSpeed[i] < 0:
                divXYZ.append(bound.lineCentres[i])
                divLinePoints.append(bound.linePoints[i])
                divSpeeds.append(bound.collisionSpeed[i])
    return np.array(divXYZ), np.array(divLinePoints), np.array(divSpeeds)

#Create new earth object
earth = Earth(
    startTime = 5,
    endTime = 0,
    deltaTime = 1)

#Create mesh objects
earthMesh = earth.getEarthMesh()
earth.setPlateData(earth.startTime)



speedTransfer, distTransfer = earth.boundaries.getTransfers()
uplift = earth.boundaries.getUplifts()
print(distTransfer)



#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars=uplift)
plotter.show()

In [ ]:
#Create new earth object
earth = Earth(
    startTime = 30,
    endTime = 0,
    deltaTime = 1)

earth.runTectonicSimulation()
earth.showEarth()

In [ ]:
earth.animate()